# Assignment 4: Neural Nets

### The problems in this assignment are based on the exercises of Chapter 11 in Data Mining for Business Analytics.

### Scenario: The goal is to predict the price of a used Toyota Corolla based on its specifications.

### Data: Consider again the data on used cars (ToyotaCorolla.csv) with 1436 records and details on 38 attributes, including Price, Age, KM, HP, and other specifications.

In [1]:
# Import Required Packages

%matplotlib inline

from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier, MLPRegressor
from dmba import classificationSummary, regressionSummary

no display found. Using non-interactive Agg backend


In [2]:
# Read in data
toyotaCorolla = pd.read_csv('dmba\ToyotaCorolla.csv')
toyotaCorolla.head()

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Parking_Assistant,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,0,0,0,1,0,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
2,3,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,0,1,0,0,0,1,0,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,1,1,0,1,0,1,0,0,0,0


### Question 1 (10 points) Fit a neural network model to the data. Note that this is a prediction problem (Price is a numerical variable and not categorical) and not classification.

#### ----use a single hidden layer with 2 nodes.
#### ----use predictors Age_08_04, KM, Fuel_Type, HP, Automatic, Doors, Quarterly_Tax, Mfr_Guarantee, Guarantee_Period, Airco, Automatic_airco, CD_Player, Powered_Windows, Sport_Model, and Tow_Bar.
#### ----remember to first scale the numerical predictor and outcome variables to a 0–1 scale and convert categorical predictors to dummies.split the data into training (60%) and validation (40%) datasets (use random_state=1).
#### ----
#### Record the RMS error for the training data and the validation data.


In [3]:
toyotaCorolla.columns

Index(['Id', 'Model', 'Price', 'Age_08_04', 'Mfg_Month', 'Mfg_Year', 'KM',
       'Fuel_Type', 'HP', 'Met_Color', 'Color', 'Automatic', 'CC', 'Doors',
       'Cylinders', 'Gears', 'Quarterly_Tax', 'Weight', 'Mfr_Guarantee',
       'BOVAG_Guarantee', 'Guarantee_Period', 'ABS', 'Airbag_1', 'Airbag_2',
       'Airco', 'Automatic_airco', 'Boardcomputer', 'CD_Player',
       'Central_Lock', 'Powered_Windows', 'Power_Steering', 'Radio',
       'Mistlamps', 'Sport_Model', 'Backseat_Divider', 'Metallic_Rim',
       'Radio_cassette', 'Parking_Assistant', 'Tow_Bar'],
      dtype='object')

In [4]:
# separate data into predictors and outcome components
input_vars = ['Age_08_04', 'KM', 'Fuel_Type', 'HP', 'Automatic', 'Doors', 'Quarterly_Tax', 
              'Mfr_Guarantee', 'Guarantee_Period', 'Airco', 'Automatic_airco', 'CD_Player',
              'Powered_Windows', 'Sport_Model', 'Tow_Bar', 'Price']
toyotaCorolla = toyotaCorolla[input_vars]
toyotaCorolla.head()

,Age_08_04,KM,Fuel_Type,HP,Automatic,Doors,Quarterly_Tax,Mfr_Guarantee,Guarantee_Period,Airco,Automatic_airco,CD_Player,Powered_Windows,Sport_Model,Tow_Bar,Price
0,23,46986,Diesel,90,0,3,210,0,3,0,0,0,1,0,0,13500
1,23,72937,Diesel,90,0,3,210,0,3,1,0,1,0,0,0,13750
2,24,41711,Diesel,90,0,3,210,1,3,0,0,0,0,0,0,13950
3,26,48000,Diesel,90,0,3,210,1,3,0,0,0,0,0,0,14950
4,30,38500,Diesel,90,0,3,210,1,3,1,0,0,1,0,0,13750


In [5]:
# convert the categorical data into dummy variables

processed = pd.get_dummies(toyotaCorolla, columns=['Fuel_Type'], drop_first=True)

outcome = 'Price'
predictors = [c for c in processed.columns if c != outcome]


X = processed[predictors]
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)
X = rescaledX

y = processed[outcome]
minY = y.min()
rangeY = (y.max() - y.min())
# Transform the actual values to range [0, 1]
y = (y - minY)/rangeY


C:\Users\jenkij\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [6]:
# split the data into training (60%) and validation (40%) datasets (use random_state=1).
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

In [7]:
# use a single hidden layer with 2 nodes
# train neural network with 2 hidden nodes
clf = MLPRegressor(hidden_layer_sizes=(2, ), activation='logistic', solver='lbfgs',
                    random_state=1)
clf.fit(train_X, train_y)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

#### The RMS error for the training data and the validation data.

In [8]:
# Record the RMS error for the training data and the validation data.
# training
print('RMS error for the training data:')
regressionSummary(train_y, clf.predict(train_X))
# validation
print('\nRMS error for the validation data:')
regressionSummary(valid_y, clf.predict(valid_X))

RMS error for the training data:

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 0.0395
            Mean Absolute Error (MAE) : 0.0290
          Mean Percentage Error (MPE) : -5.7433
Mean Absolute Percentage Error (MAPE) : 17.5263

RMS error for the validation data:

Regression statistics

                      Mean Error (ME) : 0.0023
       Root Mean Squared Error (RMSE) : 0.0383
            Mean Absolute Error (MAE) : 0.0296
          Mean Percentage Error (MPE) : -inf
Mean Absolute Percentage Error (MAPE) : inf


C:\Users\jenkij\AppData\Local\Continuum\anaconda3\lib\site-packages\dmba\metric.py:74: RuntimeWarning: divide by zero encountered in true_divide
  ('Mean Percentage Error (MPE)', 100 * sum(y_res / y_true) / len(y_res)),
C:\Users\jenkij\AppData\Local\Continuum\anaconda3\lib\site-packages\dmba\metric.py:75: RuntimeWarning: divide by zero encountered in true_divide
  ('Mean Absolute Percentage Error (MAPE)', 100 * sum(abs(y_res / y_true) / len(y_res))),


### Question 2 (10 points) Repeat the process, changing the number of hidden layers and nodes to (a) single layer with 5 nodes and (b) two layers, 5 nodes in each layer.

#### -----What happens to the RMS error for the training data as the number of layers and nodes increases?
#### -----What happens to the RMS error for the validation data?
#### -----Comment on the appropriate number of layers and nodes for this application


In [9]:
# Read in data
toyotaCorolla = pd.read_csv('dmba\ToyotaCorolla.csv')
# separate data into predictors and outcome components
input_vars = ['Age_08_04', 'KM', 'Fuel_Type', 'HP', 'Automatic', 'Doors', 'Quarterly_Tax', 
              'Mfr_Guarantee', 'Guarantee_Period', 'Airco', 'Automatic_airco', 'CD_Player',
              'Powered_Windows', 'Sport_Model', 'Tow_Bar', 'Price']
toyotaCorolla = toyotaCorolla[input_vars]

# convert the categorical data into dummy variables

processed = pd.get_dummies(toyotaCorolla, columns=['Fuel_Type'], drop_first=True)

outcome = 'Price'
predictors = [c for c in processed.columns if c != outcome]


X = processed[predictors]
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)
X = rescaledX

y = processed[outcome]
minY = y.min()
rangeY = (y.max() - y.min())
# Transform the actual values to range [0, 1]
y = (y - minY)/rangeY


C:\Users\jenkij\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [10]:
# split the data into training (60%) and validation (40%) datasets (use random_state=1).
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)
# (a) single layer with 5 nodes
# train neural network with 5 hidden nodes
clf_a = MLPRegressor(hidden_layer_sizes=(5, ), activation='logistic', solver='lbfgs',
                    random_state=1)
clf_a.fit(train_X, train_y)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

#### The RMS error for the training data and the validation data for part (a) Single Layer with 5 nodes

In [11]:
# Record the RMS error for the training data and the validation data.
# training
print('RMS error for the training data:')
regressionSummary(train_y, clf_a.predict(train_X))
# validation
print('\nRMS error for the validation data:')
regressionSummary(valid_y, clf_a.predict(valid_X))

RMS error for the training data:

Regression statistics

                      Mean Error (ME) : 0.0001
       Root Mean Squared Error (RMSE) : 0.0399
            Mean Absolute Error (MAE) : 0.0293
          Mean Percentage Error (MPE) : -5.3757
Mean Absolute Percentage Error (MAPE) : 17.6431

RMS error for the validation data:

Regression statistics

                      Mean Error (ME) : 0.0021
       Root Mean Squared Error (RMSE) : 0.0393
            Mean Absolute Error (MAE) : 0.0306
          Mean Percentage Error (MPE) : -inf
Mean Absolute Percentage Error (MAPE) : inf


C:\Users\jenkij\AppData\Local\Continuum\anaconda3\lib\site-packages\dmba\metric.py:74: RuntimeWarning: divide by zero encountered in true_divide
  ('Mean Percentage Error (MPE)', 100 * sum(y_res / y_true) / len(y_res)),
C:\Users\jenkij\AppData\Local\Continuum\anaconda3\lib\site-packages\dmba\metric.py:75: RuntimeWarning: divide by zero encountered in true_divide
  ('Mean Absolute Percentage Error (MAPE)', 100 * sum(abs(y_res / y_true) / len(y_res))),


In [12]:
# split the data into training (60%) and validation (40%) datasets (use random_state=1).
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)
# (b) two layers, 5 nodes in each layer
# train neural network with 5 hidden nodes
clf_b = MLPRegressor(hidden_layer_sizes=(5, 5, ), activation='logistic', solver='lbfgs',
                    random_state=1)
clf_b.fit(train_X, train_y)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

#### The RMS error for the training data and the validation data for part (b) Two Layers with 5 Nodes in each layer

In [13]:
# Record the RMS error for the training data and the validation data.
# training
print('RMS error for the training data:')
regressionSummary(train_y, clf_b.predict(train_X))
# validation
print('\nRMS error for the validation data:')
regressionSummary(valid_y, clf_b.predict(valid_X))

RMS error for the training data:

Regression statistics

                      Mean Error (ME) : -0.0000
       Root Mean Squared Error (RMSE) : 0.0399
            Mean Absolute Error (MAE) : 0.0291
          Mean Percentage Error (MPE) : -6.1436
Mean Absolute Percentage Error (MAPE) : 17.7094

RMS error for the validation data:

Regression statistics

                      Mean Error (ME) : 0.0029
       Root Mean Squared Error (RMSE) : 0.0398
            Mean Absolute Error (MAE) : 0.0310
          Mean Percentage Error (MPE) : -inf
Mean Absolute Percentage Error (MAPE) : inf


C:\Users\jenkij\AppData\Local\Continuum\anaconda3\lib\site-packages\dmba\metric.py:74: RuntimeWarning: divide by zero encountered in true_divide
  ('Mean Percentage Error (MPE)', 100 * sum(y_res / y_true) / len(y_res)),
C:\Users\jenkij\AppData\Local\Continuum\anaconda3\lib\site-packages\dmba\metric.py:75: RuntimeWarning: divide by zero encountered in true_divide
  ('Mean Absolute Percentage Error (MAPE)', 100 * sum(abs(y_res / y_true) / len(y_res))),


#### -----Comments on the appropriate number of layers and nodes for this application
##### The RMSE for the Validation Set of the Single Layer with 5 Nodes in part(a) is slightly lower than the RMSE for the Validation Set of the Two Layers with 5 Nodes for each Layer in part(b).  Based on this the appropriate number of layers the appropriate number of layers and nodes for this application is the model with the least amount of complexity.  For question 2 this would be the Single Layer with 5 Nodes.